In [1]:
import numpy as np
import sys
import os.path
import glob
import random
from collections import OrderedDict
from scipy.stats import lognorm, norm
from matplotlib import pyplot as plt
import mplhep as hep 
import pyhf
from matplotlib.ticker import ScalarFormatter,AutoMinorLocator
plt.style.use('loca_v_1_5/plot_style.txt')
from loca_v_1_5.loca import *
import time
plt.style.use(hep.style.CMS)

In [2]:
def DiTau_ATLAS_search(lhco_file, mTtot_min=0.0):

    process='p p --> τ+ τ-'

    search_description='''Recast of ATLAS search arXiv:2002.12223: 
    'SEARCH FOR HEAVY HIGGS BOSONS DECAYING INTO TWO TAU LEPTONS WITH
    THE ATLAS DETECTOR USING pp COLLISIONS AT SQRT(S)=13 TEV'
    Phys. Rev. Lett. 125 (2020) 051801'''

    #...Pre-cuts:
    basic_cuts = {                    
            'e'     : [15.0 , 1e+10 ,  2.47],   #[pt_min, pt_max, abs(eta_max)] 
            'mu'    : [7.0  , 1e+10 ,  2.5],
            'tau'   : [65.0 , 1e+10 ,  2.5],
            'jet'   : [20.0 , 1e+10 ,  2.5],
            'bjet'  : [20.0 , 1e+10 ,  2.5],
            'met'   : [0.0  , 1e+10 ,  1e+10],
            'photon': [0.0  , 1e+10 ,  1e+10] 
             }

    signals=[]; effs=[]; xsecs=[]

    #...Signal region defintion:
    
    Category_τhad_τhad = category(
            'nleps = 0',
            'ntaus >= 2',
            'pt(τ1)>130, pt(τ2)>65',
            'OS ττ',
            '|dphi|>2.7',
            'b-veto category', 
            'mTtot > '+str(mTtot_min))
    
    #...Observables:

    bins=[150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1150, 1500]
    mTtot_spectrum=spectrum(name='mTtot')
    
    #...Apply cuts:
    
    lhco=open_lhco(file=lhco_file, cuts=basic_cuts, lhco_header=True)

    with lhco as events:   
        for N, event in events:
            
    #.......tau_had tau_had category:
            Category_τhad_τhad.start_cutflow(name='τhad')

            if event.nleps==0:
                Category_τhad_τhad.apply_cut('nleps = 0')

                if event.ntaus>=2:
                    Category_τhad_τhad.apply_cut('ntaus >= 2')

                    τ1=event.tau_leading
                    τ2=event.tau_subleading

                    if τ1.pt>130.0 and τ2.pt>65.0 :
                        Category_τhad_τhad.apply_cut('pt(τ1)>130, pt(τ2)>65')

                        if τ1.charge==-τ2.charge: 
                            Category_τhad_τhad.apply_cut('OS ττ')

                            if abs(τ1.dphi(τ2))>2.7:
                                Category_τhad_τhad.apply_cut('|dphi|>2.7') 

                                if event.nbjets==0:
                                    Category_τhad_τhad.apply_cut('b-veto category')
                                    mT_tot=τ1.MTtot(τ2,event.met)
                                    if mT_tot>mTtot_min:
                                        Category_τhad_τhad.apply_cut('mTtot > '+str(mTtot_min))
                                        mTtot_spectrum.extract_spectrum(mT_tot)

        #search_results(Category_τhad_τhad).print(process=process, description=None, save_file=False)        
        lhco.parameters['signal_category']=Category_τhad_τhad
        lhco.parameters[mTtot_spectrum.name]=mTtot_spectrum
        lhco.parameters['mTtot_bins']=bins
        lhco.parameters['effs']=mTtot_spectrum.bin_data(bins)/float(lhco.parameters['nevents'])

    return lhco.parameters


In [3]:
def extract_results(data_files):

    FF={'FS':'scalar','FT':'tensor','FST':'scalar-tensor',
        'FVLL':'vector LL','FVLR':'vector LR','FDq':'dipole'}      
    
    kernel={}
    
    for lhco in glob.glob(data_files, recursive=True):        
        print('analyzing: '+lhco)
        res=DiTau_ATLAS_search(lhco, mTtot_min=150)
        info=res['run_tag'].split('_') 
        if 'int' in info: typ='SM*NP'
        else: typ='NP*NP' 
        if '00' in info:
            qqbar=res['run_tag'].split('_00_')[1].split('_')[0]+'~'
            
        elif '10' in info:
            qqbar=res['run_tag'].split('_10_')[1].split('_')[0]+'~'  
            
        elif '01' in info:
            qqbar=res['run_tag'].split('_01_')[1].split('_')[0]+'~'
        
        kernel[res['run_tag']]={}        
        kernel[res['run_tag']]['Form_Factor']=FF[info[0]]
        kernel[res['run_tag']]['FF_coeff']=list(info[1])
        kernel[res['run_tag']]['qq~']=qqbar
        kernel[res['run_tag']]['ll~']=res['run_tag'].split('_00_')[1].split('_')[1]
        kernel[res['run_tag']]['type']=typ
        kernel[res['run_tag']]['mll_bin']=[float(info[-2]),float(info[-1])]
        kernel[res['run_tag']]['results']=res
            
    return kernel




# def get_spectrum(run, bins=None, effs=False, luminosity=140.0):
#     norm=Results[run]['xsec']/Results[run]['nevents']    
#     if effs:
#         norm=1./Results[run]['nevents']
#     else:
#         norm=luminosity*Results[run]['xsec']/Results[run]['nevents']    
#     if bins==None:                           
#         bins=Results[run]['bins']  
#     counts=norm*Results[run]['mTtot'].bin_data(bins=bins)
#     return counts

# def unfolding_kernel_matrix(F,qq,fig=False):
#     bins=[150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1150, 1500]
#     xbins, ybins = bins, bins
#     H = [
#         list(get_spectrum(F+'_00_'+qq+'_tata_150_200', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_200_250', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_250_300', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_300_350', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_350_400', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_400_450', effs=True)),     
#         list(get_spectrum(F+'_00_'+qq+'_tata_450_500', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_500_600', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_600_700', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_700_800', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_800_900', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_900_1000', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_1000_1150', effs=True)),
#         list(get_spectrum(F+'_00_'+qq+'_tata_1150_15000', effs=True))]

#     if fig:
#         fig = plt.figure(figsize=(10,10))
#         plt.xlabel(r'$m_{\tau\tau}$')
#         plt.ylabel(r'$m_T^{\rm tot}$')
#         hep.hist2dplot(np.array(H), xbins, ybins)
#         H=None
#     return H
    

In [5]:
t0 = time.time()

FS_kernel  = extract_results(data_files='lhco_files/lhco_FS/*.gz')
FST_kernel = extract_results(data_files='lhco_files/lhco_FST/*.gz')
FT_kernel  = extract_results(data_files='lhco_files/lhco_FT/*.gz')
FVLL_kernel  = extract_results(data_files='lhco_files/lhco_FVLL/*.gz')
FVLR_kernel  = extract_results(data_files='lhco_files/lhco_FVLR/*.gz')
FD_kernel  = extract_results(data_files='lhco_files/lhco_FD/*.gz')

t1 = time.time()

print('time =',(t1-t0)/3600.)

analyzing: lhco_files/lhco_FS/FS_00_ss_tata_1150_15000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sb_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ss_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bs_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bb_tata_1150_15000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_db_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cu_tata_1150_15000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_dd_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_dd_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sd_tata_1150_15000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cc_tata_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cc_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_db_tata_1000_1150_delphes_events.lhco.gz
analyzi

analyzing: lhco_files/lhco_FS/FS_00_db_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bs_tata_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ss_tata_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_cu_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bs_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ss_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_sd_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bd_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ds_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_uu_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_ss_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_bs_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FS/FS_00_db_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/l

analyzing: lhco_files/lhco_FST/FST_00_cu_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cu_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cc_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uc_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uu_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cc_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uc_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uc_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cu_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_cc_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FST/FST_00_uu_tata_450_500_delphes_events.lhc

analyzing: lhco_files/lhco_FVLL/FVLL_00_sd_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dd_tata_Z_int_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uc_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ss_tata_A_int_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cc_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cc_tata_Z_int_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ds_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_bb_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_bb_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sb_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FV

analyzing: lhco_files/lhco_FVLL/FVLL_00_uu_tata_A_int_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uc_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cu_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ss_tata_A_int_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dd_tata_A_int_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_cu_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_dd_tata_A_int_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uu_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uu_tata_Z_int_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_ds_tata_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_uu_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FVLL/FVLL_00_sd_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/

...

analyzing: lhco_files/lhco_FD/FDq_ZZ_00_sb_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_cc_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bd_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bs_tata_1150_15000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_ds_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bb_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_sb_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_sb_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_bb_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_dd_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_db_tata_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_dd_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_dd_ta

analyzing: lhco_files/lhco_FD/FDq_AZ_00_uc_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_ss_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bs_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uc_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_uc_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_ss_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bs_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_cu_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_db_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_uc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_dd_tata_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_db_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_cc_tata_350

analyzing: lhco_files/lhco_FD/FDq_AZ_00_ds_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bd_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_sd_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_ds_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_uu_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_uu_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_cc_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_cc_tata_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_sd_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bd_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_cu_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_uc_tata_1000_1150_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uc_tata_

analyzing: lhco_files/lhco_FD/FDq_ZZ_00_bd_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_bb_tata_900_1000_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_uu_tata_350_400_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_sd_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uu_tata_450_500_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bd_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_cc_tata_400_450_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_bb_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_uu_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_sb_tata_500_600_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_uu_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_uu_tata_800_900_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uu_tata_100

analyzing: lhco_files/lhco_FD/FDq_ZZ_00_sb_tata_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_sd_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_bd_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_ds_tata_600_700_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_sd_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uu_tata_200_250_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bd_tata_250_300_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_ZZ_00_uu_tata_700_800_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_bs_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AZ_00_ss_tata_150_200_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bs_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_ss_tata_300_350_delphes_events.lhco.gz
analyzing: lhco_files/lhco_FD/FDq_AA_00_bs_tata_800_

In [12]:
def kernel_matrix(kernel,bins,output):  
    
    def get_bins(bins):
        d={}
        for i,b in enumerate(bins[:-1]):
            I=str([float(b),float(bins[i+1])])
            d[I]=i
        return d

    runs=[] 
    info={}
    for run in kernel.keys():
        x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
        if x not in runs:
            runs.append(x)
            info[x]=kernel[run]
    dic={}
    for key in runs:
        dic[key]={}
    
    bin_number=get_bins(bins)
    
    for r in runs:
        for run in kernel.keys():
            x='_'.join(kernel[run]['results']['run_tag'].split('_')[:-2])
            b=str(kernel[run]['mll_bin'])
            eff=kernel[run]['results']['effs']
            if kernel[run]['type']=='SM*NP':
                xsec=round(kernel[run]['results']['xsec'],5)
            else:
                xsec=round(2*kernel[run]['results']['xsec'],5)
            pre=[str(xsec),
                 str(kernel[run]['mll_bin'][0]),
                 str(kernel[run]['mll_bin'][1])]
            
            eff_str=''
            
            for y in eff: eff_str+=str(y)+'\t'
            dic[x][bin_number[b]]='  '.join(pre+[eff_str]) 
                       
    for d in dic.keys():
        file=open(output+'/'+d+'.dat','w')
        file.write('#===============\n')
        file.write('# ll~ : '+ info[d]['ll~'] +'\n')
        file.write('# qq~ : '+ info[d]['qq~'] +'\n')
        ff=info[d]['Form_Factor'].split()
        file.write('# FF  : '+ ff[0] +'\n')
        
        # XY:
        if len(ff)==1:    
            file.write('# XY  : LL LR RL RR\n')
        else:
            if info[d]['Form_Factor'].split()[1]=='LL':
                file.write('# XY  : LL RR  \n')
            elif info[d]['Form_Factor'].split()[1]=='LR':
                file.write('# XY  : LR RL  \n')  
                
        coeff=info[d]['FF_coeff'][0].replace("'", "")+' '+info[d]['FF_coeff'][1].replace("'", "")
                
        
        if (coeff=='0 0'or coeff=='1 0' or coeff=='0 1'):
            file.write('# coef : '+coeff+'\n')
            if '_A_int'in info[d]['results']['run_tag']:
                file.write('# type : A*Reg\n')
            elif '_Z_int'in info[d]['results']['run_tag']:
                file.write('# type : Z*Reg\n')
            else:
                file.write('# type : Reg*Reg\n')        
        else:
            file.write('# coef : 0 0 \n')
            file.write('# type : '+ '*'.join(coeff.split()) +'\n')

        
        file.write('# Nevs : '+ str(info[d]['results']['nevents'])+'\n')
        file.write('#===============\n'  )
        file.write('#\n'  )
        file.write('# XSEC\tBIN_MIN\tBIN_MAX \t Kij \n')
        file.write('#\n'  )
        
        for i in list(bin_number.values()):
            try:
                file.write(dic[d][i]+'\n')
            except KeyError:
                inv_bin_number = {v: k for k, v in bin_number.items()}
                print('Error -->',d,':',inv_bin_number[i])
                file.write((str(None)+'\t')*len(bin_number)+'\n')
        file.close()
 

In [13]:

output=make_dir('Kernel_effs', overwrite=True)

bins=[150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1150, 15000]

kernel_matrix(FS_kernel,bins,output)
kernel_matrix(FT_kernel,bins,output)
kernel_matrix(FST_kernel,bins,output)
kernel_matrix(FVLL_kernel,bins,output)
kernel_matrix(FVLR_kernel,bins,output)
kernel_matrix(FD_kernel,bins,output)

In [11]:
FD_kernel

{'FDq_AA_00_dd_tata_900_1000': {'Form_Factor': 'dipole',
  'FF_coeff': ['A', 'A'],
  'qq~': 'dd~',
  'll~': 'tata',
  'type': 'NP*NP',
  'mll_bin': [900.0, 1000.0],
  'results': {'run_tag': 'FDq_AA_00_dd_tata_900_1000',
   'nevents': 50000,
   'xsec': 0.64339,
   'signal_category': <loca_v_1_5.loca.category at 0x7fea943e2340>,
   'mTtot': <loca_v_1_5.loca.spectrum at 0x7fea943e2910>,
   'mTtot_bins': [150,
    200,
    250,
    300,
    350,
    400,
    450,
    500,
    600,
    700,
    800,
    900,
    1000,
    1150,
    1500],
   'effs': array([0.000e+00, 6.000e-05, 1.200e-03, 2.960e-03, 4.400e-03, 6.060e-03,
          8.020e-03, 1.960e-02, 2.476e-02, 2.956e-02, 2.546e-02, 1.000e-02,
          7.600e-04, 6.000e-05])}},
 'FDq_ZZ_00_sd_tata_1000_1150': {'Form_Factor': 'dipole',
  'FF_coeff': ['Z', 'Z'],
  'qq~': 'sd~',
  'll~': 'tata',
  'type': 'NP*NP',
  'mll_bin': [1000.0, 1150.0],
  'results': {'run_tag': 'FDq_ZZ_00_sd_tata_1000_1150',
   'nevents': 50000,
   'xsec': 0.0188204

In [14]:
import numpy as np
import sys
import os.path
import sys
import glob
import shlex
import shutil
from shutil import copyfile


#==========================

mll_bins=[150, 200, 250, 300, 350, 400, 450, 500, 600, 700, 800, 900, 1000, 1150, 15000]

def generate_scripts(FF, bins, Drell_Yan='NC'):
    
    l_dic={'e':'1','mu':'2','ta':'3', 've':'1', 'vm':'2', 'vt':'3' }
    q_dic={'u':'1','d':'1','c':'2', 's':'2', 'b':'3'}
    ll=''.join(dilep)
    path0=make_dir('scripts_'+ll, overwrite=True)
    
    for i,XMAX in enumerate(bins[1:]):
        XMIN=bins[i]
        path=make_dir(path0+'/scripts_SMEFT_'+ll+'_'+str(XMIN)+'_'+str(XMAX), overwrite=True)

        for ff in list(FF.keys()):
            for q in ['u','c','d','s','b']:

                temp_file='PROC_FX_NM_Q_CURRENT_TYPE_XMIN_XMAX_template'
                file=temp_file.replace('FX',ff.split('_')[0])
                file=file.replace('Q',q)
                file=file.replace('TYPE','_'.join(FF[ff]['Type']))
                file=file.replace('CURRENT',ll)
                file=file.replace('NM',FF[ff]['coeff'])
                file=file.replace('XMIN',str(XMIN))
                file=file.replace('XMAX',str(XMAX))
                file=file.split('_')[:-1]
                file='_'.join(file)

                template='templates/'+temp_file+'.txt'
                script=open(path+'/'+file,'w')
                Break=True

                with open(template) as f:

                    for line in f:    

                        line=line.replace('FR_MODEL',ufo_model)
                        line=line.replace('DELPHES_CARD_PATH',delphes_card)
                        line=line.replace('L1',dilep[0])
                        line=line.replace('L2',dilep[1])
                        line=line.replace('V1',FF[ff]['V1'])
                        line=line.replace('V2',FF[ff]['V2'])
                        line=line.replace('_Q_',q)
                        line=line.replace('CPL',FF[ff]['coupl_order'])
                        line=line.replace('TYPE','_'.join(FF[ff]['Type']))
                        line=line.replace('FX',ff.split('_')[0])
                        line=line.replace('NM',FF[ff]['coeff'])  
                        line=line.replace('_I_',q_dic[q])
                        line=line.replace('_ALPHA_',l_dic[dilep[0]])  
                        line=line.replace('_BETA_',l_dic[dilep[1]])
                        line=line.replace('XMAX',str(XMAX))
                        line=line.replace('XMIN',str(XMIN))
                        
                        if Drell_Yan=='NC':

                            line=line.replace('_Ql1_','-')
                            line=line.replace('_Ql2_','+')

                            if q in ['d','s','b']:
                                line=line.replace('_P_','d~ s~ b~')
                                if FF[ff]['WC1_d']:

                                    line=line.replace('WXC1',FF[ff]['WC1_d'][0])
                                    line=line.replace('X1',str(FF[ff]['WC1_d'][1]))
                                    line=line.replace('_Q1_','d')
                                    line=line.replace('_Q2_','s')
                                    line=line.replace('_Q3_','b')

                                    if '>>>' in line:
                                        line=line.replace('>>>','')
                                    if 'WXC2' in line:
                                        if FF[ff]['WC2_d']:
                                            line=line.replace('WXC2',FF[ff]['WC2_d'][0])
                                            line=line.replace('X2',str(FF[ff]['WC2_d'][1]))
                                        else:
                                            line=''                                
                                elif FF[ff]['coeff']=='SM':
                                    line=line.replace('_Q1_',q)
                                    if 'WXC' in line:
                                        line=''
                                    if '# run 2'in line:
                                        Break=False
                                else:                        
                                    os.remove(path+'/'+file)
                                    break

                            else:
                                line=line.replace('_P_','u~ c~')
                                if FF[ff]['WC1_u']:

                                    line=line.replace('WXC1',FF[ff]['WC1_u'][0])
                                    line=line.replace('X1',str(FF[ff]['WC1_u'][1]))
                                    line=line.replace('_Q1_','u')
                                    line=line.replace('_Q2_','c')

                                    if '>>>' in line:
                                        line=''

                                    if 'WXC2' in line:
                                        if FF[ff]['WC2_u']:
                                            line=line.replace('WXC2',FF[ff]['WC2_u'][0])
                                            line=line.replace('X2',str(FF[ff]['WC2_u'][1]))
                                        else:
                                            line=''
                                elif FF[ff]['coeff']=='SM':

                                    line=line.replace('_Q1_',q)
                                    if 'WXC' in line:
                                        line=''
                                    if '# run 2'in line:
                                        Break=False
                                else:
                                    os.remove(path+'/'+file)
                                    break

                            if 'MLLMIN' or 'MLLMAX' in line:
                                line=line.replace('MLLMIN',str(int(XMIN)))
                                line=line.replace('MLLMAX',str(int(XMAX)))
                                
                            if 'PTMIN' or 'PTMAX' in line:
                                line=line.replace('PTMIN','0.0')
                                line=line.replace('PTMAX','-1.0')

                            if Break:
                                script.write(line)
                            else:
                                script.write('')

                        elif Drell_Yan=='CC':

                            if q in ['d','s','b']:
                                line=line.replace('_P_','u~ c~')
                                line=line.replace('_Ql1_','-')
                                line=line.replace('_Ql2_','~')
                                
                                if FF[ff]['WC1_du']:

                                    line=line.replace('WXC1',FF[ff]['WC1_du'][0])
                                    line=line.replace('X1',str(FF[ff]['WC1_du'][1]))
                                    line=line.replace('_Q1_','u')
                                    line=line.replace('_Q2_','c')

                                    if '>>>' in line:
                                        line=''
                                    if 'WXC2' in line:

                                        if FF[ff]['WC2_du']:
                                            line=line.replace('WXC2',FF[ff]['WC2_du'][0])
                                            line=line.replace('X2',str(FF[ff]['WC2_du'][1]))
                                        else:
                                            line=''       

                                elif FF[ff]['coeff']=='SM':

                                    line=line.replace('_Q1_',q)
                                    if 'WXC' in line:
                                        line=''
                                    if '# run 2'in line:
                                        Break=False
                                else:                        
                                    os.remove(path+'/'+file)
                                    break

                            else:
                                line=line.replace('_P_','d~ s~ b~')
                                line=line.replace('_Ql1_','+')
                                line=line.replace('_Ql2_','')

                                if FF[ff]['WC1_ud']:
                                    line=line.replace('WXC1',FF[ff]['WC1_ud'][0])
                                    line=line.replace('X1',str(FF[ff]['WC1_ud'][1]))
                                    line=line.replace('_Q1_','d')
                                    line=line.replace('_Q2_','s')
                                    line=line.replace('_Q3_','b')

                                    if '>>>' in line:
                                        line=line.replace('>>>','')

                                    if 'WXC2' in line:
                                        if FF[ff]['WC2_ud']:
                                            line=line.replace('WXC2',FF[ff]['WC2_ud'][0])
                                            line=line.replace('X2',str(FF[ff]['WC2_ud'][1]))
                                        else:
                                            line=''

                                elif FF[ff]['coeff']=='SM':
                                    line=line.replace('_Q1_',q)
                                    if 'WXC' in line:
                                        line=''
                                    if '# run 2'in line:
                                        Break=False
                                else:
                                    os.remove(path+'/'+file)
                                    break

                            if 'PTMIN' or 'PTMAX' in line:
                                line=line.replace('PTMIN',str(int(XMIN)))
                                line=line.replace('PTMAX',str(int(XMAX)))
                                
                            if 'MLLMIN' or 'MLLMAX' in line:
                                line=line.replace('MLLMIN','0.0')
                                line=line.replace('MLLMAX','-1.0')
                                
                            if Break:
                                script.write(line)
                            else:
                                script.write('')

                    script.close()

In [16]:
AAA=3
'AAA'
eval('AAA')

3